# Prompting


In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00


In [2]:
import openai
import pandas as pd
from google.colab import userdata
from google.colab import drive
import requests

openai.api_key  = userdata.get('OPENAI_API_KEY')
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def obtener_respuesta(prompt, rol="user", modelo="gpt-3.5-turbo", temperatura=0, max_tokens=1000, frecuencia_penalizacion=0, presencia_penalizacion=0):
    """
    Obtiene una respuesta del modelo de chat especificado a partir de un mensaje con un rol dado.

    Parámetros:
    prompt (str): El mensaje que se enviará al modelo para obtener una respuesta.
    rol (str): El rol del mensaje, puede ser "user" o "system". Por defecto es "user".
    modelo (str): El nombre del modelo de chat a utilizar. Por defecto es "gpt-3.5-turbo".
    temperatura (float): Controla la aleatoriedad de la respuesta. Valores más bajos hacen respuestas más determinísticas. Por defecto es 0.
    max_tokens (int): Número máximo de tokens a generar en la respuesta. Por defecto es 150.
    frecuencia_penalizacion (float): Penalización para la frecuencia de palabras repetidas. Por defecto es 0.
    presencia_penalizacion (float): Penalización para la aparición de nuevas palabras. Por defecto es 0.

    Retorna:
    str: La respuesta generada por el modelo de chat.
    """
    # Preparamos el mensaje con el rol y el contenido del prompt
    mensajes = [{"role": rol, "content": prompt}]

    # Llamamos a la API del cliente para obtener la respuesta del modelo de chat
    respuesta = openai.ChatCompletion.create(
        model=modelo,
        messages=mensajes,
        temperature=temperatura,  # Controla la aleatoriedad de la respuesta
        max_tokens=max_tokens,    # Limita el número de tokens en la respuesta
        frequency_penalty=frecuencia_penalizacion,  # Penaliza palabras repetidas
        presence_penalty=presencia_penalizacion     # Penaliza la aparición de nuevas palabras
    )

    # Extraemos y retornamos el contenido de la respuesta generada por el modelo
    return respuesta.choices[0].message.content

In [ ]:
# Definir lista de tipos de recetas
tipo_recetas = ['chilenas',  'veganas', 'festivas', 'saludables']

frames = []

for tipo in tipo_recetas:
  print(tipo)

  # Parámetros para la solicitud
  numero_recetas = 2
  rol = "Eres un experto en cocina"
  prompt = f"{rol}. Necesito solo el nombre de {numero_recetas} recetas para el tipo de recetas: {tipo}, en una lista y formato json"

  # Obtener respuesta (suponiendo que la función 'obtener_respuesta' ya está definida)
  respuesta = obtener_respuesta(prompt)

  # Convertir respuesta a diccionario
  dct_temp = dict(eval(respuesta))

  # Crear DataFrame con las recetas
  temp = pd.DataFrame(list(dct_temp.values())[0], columns=['receta']).assign(tipo_receta=tipo)

  # Reordenar columnas del DataFrame
  temp = temp[['tipo_receta', 'receta']]

  # Agrear Recetas
  frames.append(temp)

# Juntar resultados
df_tipo_recetas = pd.concat(frames)
df_tipo_recetas

chilenas
veganas
festivas
saludables


,tipo_receta,receta
0,chilenas,Cazuela de ave
1,chilenas,Pastel de choclo
0,veganas,Hamburguesas de lentejas
1,veganas,Curry de garbanzos y espinacas
0,festivas,Pavo relleno
1,festivas,Rosca de reyes
0,saludables,Ensalada de quinoa y aguacate
1,saludables,Salmón al horno con verduras


In [ ]:
frames = []

for _,row in df_tipo_recetas.iterrows():
  print(row['tipo_receta'],row['receta'])
  tipo_receta = row['tipo_receta']
  receta = row['receta']
  prompt = f"""{rol}.
    Para la receta {receta}, quiero que me devuelvas enformato JSON,
    con las siguientes intrucciones, ademas, cada instruccion, el resultado debe ser un string:
    - Dificultad
    - Tiempo (en minutos)
    - Ingredientes (separados cada ingrediente por un -)
    - Preparacion (por pasos, y cada paso separado por un -)
  """

  # Obtener respuesta
  respuesta = obtener_respuesta(prompt, rol="user")

  # Convertir respuesta a diccionario
  dct_temp = dict(eval(respuesta))

  # Convertir el diccionario a un DataFrame
  temp = pd.DataFrame.from_dict(dct_temp, orient='index').T

  # Agregar 'tipo_receta' y 'receta'
  temp = temp.assign(Tipo_Receta=tipo_receta, Receta=receta)

  # Ordenar Columnas

  temp = temp[['Tipo_Receta', 'Receta', 'Dificultad', 'Tiempo', 'Ingredientes', 'Preparacion']]

  # Agregar a la lista
  frames.append(temp)

# Juntar resultados
df_recetas = pd.concat(frames)
df_recetas

chilenas Cazuela de ave
chilenas Pastel de choclo
veganas Hamburguesas de lentejas
veganas Curry de garbanzos y espinacas
festivas Pavo relleno
festivas Rosca de reyes
saludables Ensalada de quinoa y aguacate
saludables Salmón al horno con verduras


,Tipo_Receta,Receta,Dificultad,Tiempo,Ingredientes,Preparacion
0,chilenas,Cazuela de ave,Media,60,1 pollo troceado- 2 zanahorias- 1 cebolla- 2 d...,"1- En una cazuela grande, calentar un poco de ..."
0,chilenas,Pastel de choclo,Media,60,Choclo- Harina- Azúcar- Leche- Mantequilla- Hu...,1. Cocinar el choclo y molerlo- 2. Mezclar el ...
0,veganas,Hamburguesas de lentejas,Media,45,1 taza de lentejas cocidas - 1/2 cebolla picad...,"1- En un bol, mezclar las lentejas cocidas, la..."
0,veganas,Curry de garbanzos y espinacas,Media,45,1 lata de garbanzos- 1 cebolla- 2 dientes de a...,1- Picar la cebolla y los dientes de ajo finam...
0,festivas,Pavo relleno,Media,180,1 pavo de 5-6 kg - 200g de mantequilla - 2 ceb...,1- Precalentar el horno a 180°C. 2- Limpiar y ...


In [ ]:
images_path = '/content/drive/MyDrive/Seth-Nut/Proyectos/st_recipes/images/'
frames = []

for _,row in df_tipo_recetas.iterrows():
  print(row['tipo_receta'],row['receta'])
  tipo_receta = row['tipo_receta']
  receta = row['receta']

  # Define el prompt para la imagen
  prompt = f"Una receta de {receta}"

  # Genera la imagen usando DALL·E 2
  response = openai.Image.create(
      prompt=prompt,
     n=1,  # Número de imágenes a generar
     size="512x512"  # Tamaño de la imagen
  )

  # Obtén la URL de la imagen generada
  image_url = response['data'][0]['url']

  # Realiza una solicitud GET a la URL de la imagen
  response = requests.get(image_url)

  # Verifica que la solicitud fue exitosa
  if response.status_code == 200:
    # Abre un archivo en modo binario para escribir la imagen
    with open(images_path + f'{receta}.png', 'wb') as file:
      file.write(response.content)
      print("Imagen descargada y guardada como f'{receta}.png'")
  else:
      print("Error al descargar la imagen:", response.status_code)

  # Agregar a la lista
  frames.append(f'{receta}.png')

chilenas Cazuela de ave
Imagen descargada y guardada como f'{receta}.png'
chilenas Pastel de choclo
Imagen descargada y guardada como f'{receta}.png'
veganas Hamburguesas de lentejas
Imagen descargada y guardada como f'{receta}.png'
veganas Curry de garbanzos y espinacas
Imagen descargada y guardada como f'{receta}.png'
festivas Pavo relleno
Imagen descargada y guardada como f'{receta}.png'
festivas Rosca de reyes
Imagen descargada y guardada como f'{receta}.png'
saludables Ensalada de quinoa y aguacate
Imagen descargada y guardada como f'{receta}.png'
saludables Salmón al horno con verduras
Imagen descargada y guardada como f'{receta}.png'


In [ ]:
frames

['Cazuela de ave.png',
 'Pastel de choclo.png',
 'Hamburguesas de lentejas.png',
 'Curry de garbanzos y espinacas.png',
 'Pavo relleno.png',
 'Rosca de reyes.png',
 'Ensalada de quinoa y aguacate.png',
 'Salmón al horno con verduras.png']

In [ ]:
# Guardar Resultados
data_path = '/content/drive/MyDrive/Seth-Nut/Proyectos/st_recipes/data/'
df_recetas['images'] = frames
df_recetas.to_csv(data_path + 'recetas.csv', index=False)